<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ufidon/nlp/blob/main/qair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ufidon/nlp/blob/main/qair.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>
<br>

**Question Answering, Information Retrieval, and Retrieval Augmented Generation**

- 📝 SALP chapter 14

## Overview
- `Question answering (QA) systems` have supported human information needs since the 1960s, advancing over time with innovations like IBM’s Watson, which surpassed human performance on "Jeopardy!" in 2011.
  - They are integrated with search engines and modern large language models (LLMs), enhancing their ability to answer diverse queries.

- `Prompt-based methods` in modern QA use pretrained LLMs to directly answer fact-based questions from stored knowledge.
  - LLMs often produce `hallucinations`, confidently giving incorrect answers, especially in specialized domains, due to poor calibration.
  - `Simple prompting LLMs` limits their ability to access proprietary or real-time data, as they cannot update information after training.
  - `Retrieval-augmented generation (RAG)` overcomes these issues by combining `document retrieval` with LLMs, grounding answers in relevant, curated data and enabling responses based on proprietary information.
  - RAG uses `information retrieval techniques`, from tf-idf to neural models like BERT, to select relevant documents, enhancing answer reliability and context-awareness.

## Information Retrieval
- `Information retrieval (IR)` is the field focused on retrieving media based on user needs,
  -  with IR systems often functioning as `search engines`; 
  -  `ad hoc retrieval` is a core IR task where users enter queries to find ordered sets of relevant documents.

![The architecture of an ad hoc IR system](./images/qa/ir.png)

- Key components of IR include 
  - documents: text units like web pages or articles, 
  - collections: sets of documents, 
  - terms: words or phrases in documents, and 
  - queries: user-expressed needs.

- IR systems commonly use the `vector space` model, 
  - representing documents and queries as vectors based on word counts and ranking them by cosine similarity, 
  - following a bag-of-words approach where word positions are disregarded.

### Term weighting and document scoring
- IR scores a document’s match with a query by assigning term weights to each word, commonly using weighting schemes:
  - [tf-idf](https://scikit-learn.org/1.5/modules/feature_extraction.html): term-frequency — inverse document-frequency
  - [BM25](https://huggingface.co/blog/xhluca/bm25s): Best Match 25

- **Tf-idf** is the product of term frequency (**tf**) and inverse document frequency (**idf**): 
  - **Term frequency (tf)** measures word frequency in a document, using a logarithmic scale to avoid overemphasis on high counts:
    - $tf_{t, d} = 
    \begin{cases} 
      1 + \log_{10}(\text{count}(t, d)) & \text{if count}(t, d) > 0 \\
      0 & \text{otherwise}
   \end{cases}$

  - **Inverse Document Frequency (idf)** measures how informative a term is across documents, `decreasing` for terms appearing in many documents:
    - $idf_t = \log_{10} \dfrac{N}{df_t}$
    where $N$ is the total number of documents, and $df_t$ is the number of documents containing term $t$.
- The **tf-idf** score for a word $t$ in document $d$ is computed by multiplying `tf` and `idf`:
  - $\text{tf-idf}(t, d) = tf_{t, d} \cdot idf_t$

### Document scoring
- Document scoring uses the **cosine similarity** between the query vector $𝐪$ and document vector $𝐝$, computed as:
  - $\text{score}(𝐪, 𝐝) = \cos(𝐪, 𝐝) = \dfrac{𝐪 \cdot 𝐝}{|𝐪||𝐝|}$

  - Cosine similarity can also be viewed as the **dot product of unit vectors** by normalizing both vectors to unit length before calculation:
    - $\text{score}(𝐪, 𝐝) = \cos(𝐪, 𝐝) = \left(\dfrac{𝐪}{|𝐪|}\right) \cdot \left(\dfrac{𝐝}{|𝐝|}\right)$
  - In `tf-idf`:
    - $\text{score}(𝐪, 𝐝) = \sum_{t \in 𝐪} \frac{\text{tf-idf}(t, 𝐪)}{\sqrt{\sum_{q_i \in 𝐪} \text{tf-idf}^2(q_i, 𝐪)}} \cdot \frac{\text{tf-idf}(t, 𝐝)}{\sqrt{\sum_{d_i \in 𝐝} \text{tf-idf}^2(d_i, 𝐝)}}$

- Simplifications and **variants of tf-idf cosine** scoring are used in practice, such as omitting the **idf term** in the document to enhance performance.

  - **BM25** is an advanced variant of tf-idf that introduces two parameters
    - **k** (balancing term frequency and IDF) 
    - **b** (adjusting document length normalization)—to refine scoring.

- **Stop lists** (lists of high-frequency words like "the," "a") were traditionally used to exclude common terms from queries and documents, reducing index size; 
  - modern IR systems now rarely use stop lists due to advancements in IDF weighting and processing efficiency.


### [Inverted Index](https://nlp.stanford.edu/IR-book/html/htmledition/a-first-take-at-building-an-inverted-index-1.html)
- In information retrieval, the goal is to find documents containing query terms, ignoring those with none of the terms.

- An `inverted index` structure, with a dictionary and postings lists, efficiently identifies relevant documents and stores term frequencies and positions.

- The dictionary links terms to postings lists, which provide document IDs and term-related data for fast score computation.

- Alternatives, such as bigram indexing and hashing, can improve efficiency in tasks like finding Wikipedia pages for question answering.

## Information Retrieval with Dense Vectors
- Traditional IR methods like tf-idf and BM25 have the **Vocabulary Mismatch Problem**
  - They require `exact word overlap` between query and document, which limits retrieval when `synonyms` are used.

  - `Dense embeddings`, such as those generated by BERT, address this issue by encoding semantic meaning, rather than relying on exact word matches.

- **Single encoder (a)**: The query and document are fed into a BERT model, with a linear layer on the [CLS] token to predict similarity, allowing context-sensitive matching.
  - $\text{score}(𝐪,𝐝) = \text{softmax}(𝐔(𝐳))$
    - $𝐳 = \text{BERT}(𝐪;[\text{SEP}];𝐝)[\text{CLS}]$
  - Documents are split into short passages (e.g., 100 tokens) to fit within BERT's 512-token limit, and the model is fine-tuned on a relevance dataset for better retrieval accuracy.
  - `Single encoder` uses a `full BERT encoder` for each query-document pair, which is computationally expensive, 
    - as it requires encoding each document in the collection with every new query.

![Two ways to do dense retrieval](./images/qa/qdbert.png)


- **Bi-Encoder (b)** encodes each document only once and precomputes document vectors, enabling `fast` query processing by computing dot products between the query vector and precomputed document vectors.
  - $\text{score}(𝐪,𝐝) = 𝐳_𝐪 ⋅ 𝐳_𝐝$
    - $𝐳_𝐪 = \text{BERT}_Q(𝐪)[\text{CLS}]$
    - $𝐳_𝐝 = \text{BERT}_D(𝐝)[\text{CLS}]$
  - However, it sacrifices some accuracy by not capturing detailed token-level interactions between the query and document.

- Numerous approaches between the full encoder and the bi-encoder balance accuracy and efficiency by 
  - using cheaper ranking methods (e.g., BM25) to initially rank documents 
  - then applying costly BERT scoring to rerank only the top-ranked ones.

- [ColBERT (Contextualized Late Interaction over BERT)](https://huggingface.co/colbert-ir/colbertv2.0) encodes the query and document separately into token-level representations, allowing it to pre-store document representations for faster scoring.

- ![A sketch of the ColBERT algorithm at inference time](./images/qa/colbert.png)

  - It computes relevance by summing the maximum similarity between each query token and the most contextually similar document token, highlighting `token-level contextual similarity`.
  - Query and document tokens are processed with BERT, including special `[Q]` and `[D]` tokens, and then scaled to unit length, optimizing vector size for storage efficiency.
  - It requires end-to-end training, fine-tuning BERT encoders and linear layers using positive and negative document pairs, optimizing relevance scoring through cross-entropy loss.

- Training data for supervised methods like ColBERT often includes labeled positive and negative passages, with semi-supervised approaches or iterative methods used when labeled data is sparse.

- Efficient ranking of dense vectors uses `approximate nearest neighbor search algorithms` (e.g., [Faiss](https://huggingface.co/docs/datasets/en/faiss_es)) to quickly find the most similar document vectors for a query.

## Answering Questions with Retrieval-Augmented Generation
- The retrieval-augmented generation (RAG) approach to question answering involves 
  - retrieving relevant text segments (retriever) 
  - generating an answer based on these documents (reader).

![Retrieval-based question answering](./images/qa/rag.png)

- This two-stage model first uses dense retrievers to find supportive passages and then employs a large language model to generate the answer from the retrieved content, one token at a time.

### Retrieval-Augmented Generation
- RAG uses retrieved passages to help a language model generate answers, addressing limitations of simple conditional generation.
  - simple autoregressive language modeling: 
    - $\displaystyle p(x_1,⋯,x_n)=∏_{i=1}^n p([Q:]; q; [A:]; x_{<i})$ 
- In RAG, a language model is conditioned on both the question and retrieved passages to reduce issues like hallucinations and provide evidence-based answers.
  - Simple prompting can work for basic fact-based questions, but RAG `improves reliability` by using `specific prompts` like "Based on these texts, answer this question."
  - $\displaystyle p(x_1,⋯,x_n)=∏_{i=1}^n p(x_i|R(q);\text{prompt};[Q:]; q; [A:]; x_{<i})$ 
- Effective RAG requires a well-performing retriever, often utilizing a `two-stage process` that ranks retrieved passages to improve accuracy.
  - For complex questions, RAG may use `multi-hop retrieval`, combining initial retrieval results with follow-up searches for more context.
- `Privacy and prompt engineering` are important considerations, especially in combining private and public data, with ongoing research focusing on refining the integration of retrieval and generation stages.

### Question Answering Datasets
- `Question-answering (QA) datasets` support both training and evaluating language models.  They vary by purpose, 
  - targeting natural information-seeking `questions`
  - `probing` system knowledge and reasoning.
- Natural question datasets include 
  - [Natural Questions](https://huggingface.co/datasets/google-research-datasets/natural_questions): based on Google queries, with answers derived from Wikipedia
  - [MS MARCO (Microsoft Machine Reading Comprehension)](https://huggingface.co/datasets/microsoft/ms_marco): Bing queries with human-generated answers and passages.
- Non-English natural question datasets include 
  - [DuReader](https://paperswithcode.com/dataset/dureader): Chinese search engine queries
  - [TyDi QA](https://huggingface.co/datasets/google-research-datasets/tydiqa): questions in 11 languages from Wikipedia passages.
- Probing datasets, such as [MMLU (Massive Multitask Language Understanding)](https://paperswithcode.com/dataset/mmlu), assess knowledge across multiple fields like medicine and computer science, using questions sourced from exams.
- Some datasets augment questions with passages, enabling reading comprehension tasks that require extracting answers from provided texts.

- QA tasks vary: 
  - `open book` QA uses retrieval-augmented generation, 
  - `closed book` QA answers directly from the model without retrieval.
- Answer formats differ, including multiple-choice and freeform, affecting model requirements for generating or selecting responses.

- Prompting styles in QA, impacting model performance on complex questions, can be 
  - zero-shot (only the question) 
  - few-shot (with examples).
- `MMLU` supports both zero-shot and few-shot prompting, making it suitable for testing various model capabilities.

## Evaluating Question Answering
- **Exact Match**: For multiple-choice questions (like in MMLU), QA systems are evaluated by the `percentage of answers that exactly match the correct answer`.

- **Token F1 Score**: For free-text questions (e.g., Natural Questions), token F1 is used to measure `partial overlap between the predicted answer and the reference`, treating them as bags of tokens and averaging scores across all questions.

- **Mean Reciprocal Rank (MRR)**: For systems providing ranked answers, MRR evaluates the rank of the first correct answer, scoring each question based on `the reciprocal of this rank` and averaging scores across questions.
  - If no correct answer is given, the score for that question is zero, with some versions of MRR excluding these zero-score questions from the final calculation.

-🏃 Practice [Question answering from HuggingFace NLP](https://huggingface.co/learn/nlp-course/en/chapter7/7?fw=pt)

In [ ]:
# 1. Install required libraries
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

In [ ]:
# 2. [SQuAD2.0](https://rajpurkar.github.io/SQuAD-explorer/)
from datasets import load_dataset
raw_datasets = load_dataset("squad")

In [ ]:
# features of the dataset
raw_datasets

In [ ]:
# 3. check the first element
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])
# In the Answer, The text field is rather obvious, 
# and the answer_start field contains the `starting character index` of each answer in the context.

In [ ]:
# there is only one possible answer in each training record
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

In [ ]:
# but there may be several answers in each evaluation record
print(raw_datasets["validation"][0]["context"])
print(raw_datasets["validation"][0]["question"])
print(raw_datasets["validation"][0]["answers"])

In [ ]:
print(raw_datasets["validation"][2]["context"])
print(raw_datasets["validation"][2]["question"])
print(raw_datasets["validation"][2]["answers"])

In [ ]:
# 4. Processing the training data
# 4.1 prepare a [fast tokenizer](https://huggingface.co/docs/transformers/index)
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# make sure the tokenizer is really fast
tokenizer.is_fast

In [ ]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

# 4.2 The tokenizer will properly insert the special tokens to form a sentence like this:
# [CLS] question [SEP] context [SEP]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])